In [1]:
import pandas as pd
import matplotlib as plt
%matplotlib inline
import numpy as np

### Build feature vectors and label vectors - ledengre polynomial coeffs and probabilty features

In [2]:
all_train_probs = pd.read_csv("c:/Users/Rachel/Documents/Grad_Stuff/Dissertation/degree_trials_data/final_probs_7deg.csv")

In [3]:
all_train_data = pd.read_csv("c:/Users/Rachel/Documents/Grad_Stuff/Dissertation/degree_trials_data/degree7_train_normspkrmean_legens.csv")

In [4]:
all_train_probs = all_train_probs.rename(index=float, columns={"English: WH": "EnglishWH", "English: Y/N":"EnglishYN","Mandarin: WH": "MandarinWH", "Mandarin: Y/N":'MandarinYN'})

In [5]:
prob_features = all_train_probs.drop(["id", "language","questions"], axis=1)

In [ ]:
prob_features

In [6]:
all_train_data = pd.concat([all_train_data, prob_features], axis=1)

In [ ]:
all_train_data

In [7]:
coeffs = []

for i, row in all_train_data.iterrows():
    leges = np.fromstring(row["legen_coefs"][1:-1], dtype=float, sep=",")
    coeffs.append(leges)


In [ ]:
coeffs

In [ ]:
[[f'lg_{n}' for n in range(7)]]

In [8]:
####CHANGE DEGREE####range(x)where x is degree
df_coeff = pd.DataFrame(coeffs, columns = [f'lg{n}' for n in range(7)])

In [ ]:
df_coeff

In [ ]:
all_train_data

In [9]:
###rewrite this at some point, adding in these features just to get rid of them is stupid
df_coeff[['language', 'questions']] = all_train_data[['language', 'questions']]

In [10]:
df_coeff = pd.concat([df_coeff, prob_features], axis=1)

In [ ]:
#### do not need?  Unless we are counting something?
df_coeff_sorted = df_coeff.groupby(['language', 'questions'])

df_coeff_sorted.groups

df_coeff.columns

In [11]:
df_labels = pd.DataFrame({'labels':df_coeff['language'] + " " + df_coeff['questions']})

In [ ]:
df_labels

In [12]:
df_data = df_coeff.drop(['language', 'questions'], axis=1)

In [ ]:
df_data

### And here, we change to language / question subsets, to see if we are doing well

In [ ]:
df_coeff

In [ ]:
df_lang = df_coeff.drop(['language','questions'], axis=1)

In [ ]:
lang_labels = df_coeff["language"]

In [ ]:
lang_labels

In [ ]:
df_ques = df_coeff.drop(['language','questions'], axis=1)

In [ ]:
ques_labels = df_coeff['questions']

In [ ]:
ques_labels

#### add in f0 features

In [ ]:
all_train_data

In [ ]:
### make sure to deal with negative numbers?  or will this not work because of negative numbers?
all_f0s = []
for i, row in all_train_data.iterrows():
    f0s = np.fromstring(row["f0final"][1:-2], dtype=float, sep="\n")
    #print (type((f0s)))
    all_f0s.append(f0s)

In [ ]:
print(all_f0s[0])

In [ ]:
maxf0s = []
minf0s = []
for listf0s in all_f0s:
    high = max(listf0s)
    low = min(listf0s)
    maxf0s.append(high)
    minf0s.append(low)


In [ ]:
rangef0s = []
for i in range(len(maxf0s)):
    diff = maxf0s[i] - minf0s[i]
    rangef0s.append(diff)
print (len(rangef0s))

In [ ]:
df_data = pd.DataFrame({'maxf0s': maxf0s, 'minf0s':minf0s, 'rangef0s':rangef0s})
df_lang = pd.DataFrame({'maxf0s': maxf0s, 'minf0s':minf0s, 'rangef0s':rangef0s})
df_ques = pd.DataFrame({'maxf0s': maxf0s, 'minf0s':minf0s, 'rangef0s':rangef0s})

### Machine learning time!

In [13]:
import sklearn

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn import svm
from sklearn import dummy

In [15]:
from sklearn.dummy import DummyClassifier
dclf = DummyClassifier()

#### following code, amend depending on whether it's 4-way, or lang, ques

In [ ]:
df_labels

In [16]:
df_labels["labels"].value_counts() /len(df_coeff)

Mandarin WH     0.287037
English WH      0.285613
Mandarin Y/N    0.216524
English Y/N     0.210826
Name: labels, dtype: float64

In [17]:
baseline_scores = cross_val_score(dclf, df_data, df_labels.values.squeeze(), cv=10)
baseline_scores.mean()

0.2537207318451403

In [18]:
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC


C:\Users\Rachel\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Let's do some motherfuckin' GridSearching of parameters

In [ ]:
## NOTE: grid_search is depreciating, will change to model_selection module
#param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
#              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}



param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

from sklearn.grid_search import GridSearchCV
#from sklearn.svm import SVC
grid_search = GridSearchCV(LinearSVC(), param_grid, cv=10)

X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels.values.squeeze(), random_state=0)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [19]:
clf = LinearSVC(penalty='l2', C=0.01, multi_class= "ovr")
scores = cross_val_score(clf, df_data, df_labels.values.squeeze(), cv=10)

In [20]:
scores.mean()

0.31039309493154865

In [ ]:
cross_val_score?

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels, random_state=0)

#logreg = LogisticRegression().fit(X_train, y_train)
#score = logreg.score(X_test, y_test)

#classifier = linear_model.LogisticRegression()
classifier = svm.LinearSVC(penalty="l2", C=0.01, multi_class='ovr')
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
predicts = classifier.predict(X_test)

print (score, predicts)

C:\Users\Rachel\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.34472934472934474 ['English Y/N' 'English WH' 'Mandarin WH' 'Mandarin Y/N' 'Mandarin WH'
 'English WH' 'English Y/N' 'English WH' 'Mandarin WH' 'English Y/N'
 'Mandarin WH' 'English WH' 'Mandarin Y/N' 'Mandarin WH' 'Mandarin WH'
 'Mandarin Y/N' 'Mandarin WH' 'Mandarin WH' 'Mandarin WH' 'Mandarin WH'
 'Mandarin Y/N' 'Mandarin WH' 'Mandarin WH' 'English WH' 'English WH'
 'English Y/N' 'Mandarin WH' 'English WH' 'English Y/N' 'English WH'
 'Mandarin Y/N' 'Mandarin WH' 'English WH' 'Mandarin Y/N' 'Mandarin WH'
 'Mandarin Y/N' 'English WH' 'Mandarin WH' 'English WH' 'Mandarin WH'
 'English WH' 'English Y/N' 'English WH' 'English WH' 'English Y/N'
 'Mandarin WH' 'Mandarin Y/N' 'English WH' 'Mandarin WH' 'Mandarin WH'
 'English WH' 'Mandarin WH' 'Mandarin WH' 'Mandarin Y/N' 'English WH'
 'English WH' 'Mandarin WH' 'English WH' 'Mandarin WH' 'Mandarin WH'
 'Mandarin WH' 'English WH' 'Mandarin WH' 'English WH' 'English WH'
 'English WH' 'Mandarin WH' 'Mandarin WH' 'English WH' 'English WH'
 '

In [22]:
from sklearn.utils import check_X_y

In [23]:
#### BETTER THAN CROSS VALIDATION DONE THE OTHER WAY ##########
########## UPDATE PARAMATERS BASED ON ABOVE ##########
X_train, X_test, y_train, y_test = train_test_split(df_data, df_labels, random_state=0)


#classifier = DummyClassifier()
#classifier.fit(X_train, y_train)


dummy = DummyClassifier(strategy='stratified')

X_converted, y_converted = check_X_y(X=X_test, y=y_test)
dummy.fit(X=X_converted, y=y_converted)

dummy_pred = dummy.predict(X_converted)


#d_score = classifier.score(X_test, y_test)
#d_predicts = classifier.predict(X_test)

print (dummy_pred)

['Mandarin WH' 'English Y/N' 'English Y/N' 'English Y/N' 'Mandarin WH'
 'English WH' 'Mandarin WH' 'Mandarin WH' 'English WH' 'Mandarin WH'
 'Mandarin WH' 'Mandarin Y/N' 'English WH' 'Mandarin WH' 'English WH'
 'English WH' 'English Y/N' 'English Y/N' 'English WH' 'English WH'
 'Mandarin WH' 'Mandarin Y/N' 'Mandarin Y/N' 'English WH' 'Mandarin WH'
 'English Y/N' 'English Y/N' 'English Y/N' 'Mandarin WH' 'Mandarin WH'
 'English WH' 'English Y/N' 'Mandarin WH' 'Mandarin WH' 'Mandarin Y/N'
 'English WH' 'Mandarin WH' 'Mandarin WH' 'English WH' 'English WH'
 'English Y/N' 'Mandarin Y/N' 'Mandarin WH' 'English Y/N' 'English WH'
 'English WH' 'Mandarin WH' 'Mandarin WH' 'English WH' 'Mandarin Y/N'
 'English WH' 'English Y/N' 'Mandarin WH' 'English Y/N' 'English Y/N'
 'English WH' 'English WH' 'Mandarin WH' 'English WH' 'Mandarin WH'
 'English Y/N' 'English WH' 'Mandarin WH' 'Mandarin WH' 'English Y/N'
 'Mandarin Y/N' 'English Y/N' 'Mandarin Y/N' 'English Y/N' 'English Y/N'
 'Mandarin WH' 'Ma

C:\Users\Rachel\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### In which we do some math for significance testing

In [ ]:
predicts

In [24]:
gold_stand = list(df_labels["labels"])

In [ ]:
print(gold_stand)

In [25]:
all_predicts = zip(dummy_pred, predicts, gold_stand)

In [ ]:
print(list(all_predicts))

In [26]:
base_wins = 0
model_wins = 0

for(base, model, gold) in zip(dummy_pred, predicts, gold_stand):
    if base == gold:
        base_wins = base_wins+1
    if model == gold:
        model_wins = model_wins+1

print("base_wins: ", base_wins)
print("model_wins: ", model_wins)

base_wins:  95
model_wins:  83


In [27]:
### Kyle Gorman's code for McNemar's p values test ######
import scipy.stats


def mcnemar_p(b, c):
  """Computes McNemar's test.
  Args:
    b: the number of "wins" for the first condition.
    c: the number of "wins" for the second condition.
  Returns:
    A p-value for McNemar's test.
  """
  n = b + c
  x = min(b, c)
  dist = scipy.stats.binom(n, .5)
  return 2. * dist.cdf(x)


def mcnemar_midp(b, c):
  """Computes McNemar's test using the "mid-p" variant.
  This is based closely on:
    
  M.W. Fagerland, S. Lydersen, P. Laake. 2013. The McNemar test for 
  binary matched-pairs data: Mid-p and asymptotic are better than exact 
  conditional. BMC Medical Research Methodology 13: 91.
  Args:
    b: the number of "wins" for the first condition.
    c: the number of "wins" for the second condition.
  Returns:
    A p-value for the mid-p variant of McNemar's test.
  """
  x = min(b, c)
  dist = scipy.stats.binom(n, .5)
  return mcnemar_p(b, c) - dist.pmf(x)

In [28]:
result = mcnemar_p(base_wins, model_wins)
result

0.4097360735808759

#### Congratulations, your results are not at all significant!

In [ ]:
df_coeff['label'] = df_labels

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,5))
df_coeff.groupby('label').mean().T.plot.bar(ax=ax1)
df_coeff.groupby('label').std().T.plot.bar(ax=ax2)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
_ = df_coeff.boxplot(by = 'label', ax=ax, layout = (2,4))

In [ ]:
import seaborn as sns

In [ ]:
fig, ax= plt.subplots(nrows=2, ncols=4, figsize=(20,10))
axlist = ax.flatten()
for i, l in enumerate(df_coeff.columns[0:7]):
    _ = sns.boxplot(x="label", y=l, data=df_coeff, ax=axlist[i])